# SKA SDP Deploy Tutorial

# Install Extra Package

In [3]:
!pip install ska_ser_logging ska-sdp-cbf-emulator pyping
!sudo apt install -y wget iputils-ping

# To perform additional developement operations you must additional provide 
# a kubeconfig file which is by default located at:
# $HOME/.kube/config
# or KUBECONFIG environment variable 
# or can be passed to load_kube_config()

if development:
    # Developer Only Packages

    # Kubernetes
    !pip install kubernetes
    # Alternatively debug using kubectl commands
    !curl -LO "https://dl.k8s.io/release/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl"
    !sudo install -o root -g root -m 0755 kubectl /usr/local/bin/kubectl
    !kubectl --help


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artefact.skao.int/repository/pypi-internal/simple, https://pypi.org/simple
  Preparing metadata (setup.py) ... done
  Created wheel for pyping: filename=pyping-0.0.6-py3-none-any.whl size=9851 sha256=05b9389a01932fdf867749f629ff690e2cbfd16a43cfcabd953c74e69044685d
  Stored in directory: /home/tango/.cache/pip/wheels/e2/fb/bc/e3d112816c8d0422f064b074cb3afed3111b5d3c0af611092c
Successfully built pyping
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libcap2 libcap2-bin libpam-cap libpcre2-8-0
The following NEW packages will be installed:
  iputils-ping libcap2 libcap2-bin libpam-cap libpcre2-8-0 wget
0 upgraded, 6 newly installed, 0 to remove and 0 not upgraded.
Need to get 1219 kB of archives.
After this operation, 4244 kB of additional disk space will be used.
Get:1 http://deb.deb

In [5]:
if development:
    # Helm
    !sudo apt install -y gpg
    !curl https://baltocdn.com/helm/signing.asc | gpg --dearmor | sudo tee /usr/share/keyrings/helm.gpg > /dev/null
    !sudo apt-get install apt-transport-https --yes
    !echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/helm.gpg] https://baltocdn.com/helm/stable/debian/ all main" | sudo tee /etc/apt/sources.list.d/helm-stable-debian.list
    !sudo apt-get update
    !sudo apt-get install helm
    !helm --help

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  dirmngr gnupg gnupg-l10n gnupg-utils gpg-agent gpg-wks-client gpg-wks-server
  gpgconf gpgsm gpgv libassuan0 libksba8 libnpth0 pinentry-curses
Suggested packages:
  dbus-user-session libpam-systemd pinentry-gnome3 tor parcimonie xloadimage
  scdaemon pinentry-doc
The following NEW packages will be installed:
  dirmngr gnupg gnupg-l10n gnupg-utils gpg gpg-agent gpg-wks-client
  gpg-wks-server gpgconf gpgsm libassuan0 libksba8 libnpth0 pinentry-curses
The following packages will be upgraded:
  gpgv
1 upgraded, 14 newly installed, 0 to remove and 22 not upgraded.
Need to get 7674 kB of archives.
After this operation, 14.9 MB of additional disk space will be used.
Get:1 http://deb.debian.org/debian buster/main amd64 gpgv amd64 2.2.12-1+deb10u2 [584 kB]
Get:2 http://deb.debian.org/debian buster/main amd64 libassuan0 amd64 2.5.2-1 [49.4 kB]
Ge

# Connect to SDP

The SDP is a helm chart in https://gitlab.com/ska-telescope/sdp/ska-sdp-integration deployed to a namespace on the kubernetes cluster. `dp-shared` is the name of the persistent SDP namespace. For testing the following script can alternatively deploy to a custom personal or team namespace.

**Note:** When deploying the SDP the installation may take a few minutes.

In [ ]:
# Restart Kernel to refresh environment variables
import os
os._exit(00)

In [13]:
development = True
debug = True
shared = False  # use the shared persistent sdp deployment
deploy_sdp = True  # if not shared, also deploy sdp to a custom namespace
dev_sdp = True  # use the latest unreleased sdp

In [15]:
!KUBECONFIG={KUBECONFIG} helm uninstall persistent-sdp --namespace {KUBE_NAMESPACE} --wait

release "persistent-sdp" uninstalled


In [18]:
from tango import DeviceProxy, EventType
import ska_sdp_config
import os
import json
import random
from datetime import date
import logging
import ska_ser_logging

if development:
    import kubernetes
    KUBECONFIG = "/app/config" # or "$HOME/.kube/config"
    k8s_client = kubernetes.client.api_client.ApiClient(configuration=kubernetes.config.load_kube_config(KUBECONFIG))
    k8s_core = kubernetes.client.CoreV1Api()
    k8s_batch = kubernetes.client.BatchV1Api()

if shared:
    # Make sure you connect to the correct Configuration Database
    KUBE_NAMESPACE = "dp-shared"
    KUBE_PROC_NAMESPACE = f"{KUBE_NAMESPACE}-p"
else:
    KUBE_NAMESPACE = "dp-yanda-callan"  # add the namespace you want to connect to here
    KUBE_PROC_NAMESPACE = f"{KUBE_NAMESPACE}-p"
    # deploy the sdp
    if development and deploy_sdp:
        if not dev_sdp:
            !helm repo add ska https://artefact.skao.int/repository/helm-internal
            !KUBECONFIG={KUBECONFIG} helm uninstall persistent-sdp --namespace {KUBE_NAMESPACE} --wait
            !KUBECONFIG={KUBECONFIG} helm upgrade --install persistent-sdp ska/ska-sdp --namespace {KUBE_NAMESPACE} --set helmdeploy.namespace={KUBE_PROC_NAMESPACE} --wait
        else:
            # Alternative: sdp install from local repo
            !sudo apt install -y git
            !git clone https://gitlab.com/ska-telescope/sdp/ska-sdp-integration.git --recurse-submodules
            !cd ska-sdp-integration/charts/ska-sdp && helm dependency build
            !KUBECONFIG={KUBECONFIG} make uninstall-sdp -C ./ska-sdp-integration KUBE_NAMESPACE={KUBE_NAMESPACE} KUBE_NAMESPACE_SDP={KUBE_PROC_NAMESPACE}
            !KUBECONFIG={KUBECONFIG} make install-sdp -C ./ska-sdp-integration KUBE_NAMESPACE={KUBE_NAMESPACE} KUBE_NAMESPACE_SDP={KUBE_PROC_NAMESPACE}
    
# set the name of the databaseds service
DATABASEDS_NAME = "databaseds-tango-base"

# finally set the predetermined host name
os.environ["SDP_CONFIG_HOST"] = f"ska-sdp-etcd-client.{KUBE_NAMESPACE}"
os.environ["TANGO_HOST"] = f"{DATABASEDS_NAME}.{KUBE_NAMESPACE}.svc.cluster.local:10000"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.20.1-2+deb10u3).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
fatal: destination path 'ska-sdp-integration' already exists and is not an empty directory.
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "ska" chart repository
Update Complete. ⎈Happy Helming!⎈
Saving 1 charts
Deleting outdated charts
make: Entering directory '/app/ska-sdp-integration'
helm uninstall test -n dp-yanda-callan --wait
release "test" uninstalled
make: Leaving directory '/app/ska-sdp-integration'
make: Entering directory '/app/ska-sdp-integration'
# run `make install-sdp HELM_VALUES=resources/itango.yaml` to install SDP with itango
helm upgrade --install test charts/ska-sdp -n dp-yanda-callan --set helmdeploy.namespace=dp-yanda-callan-p --values resources/test.yaml --wait
Release "test" does not exist.

# Check SDP deployment

In [17]:
if development and debug:
    !KUBECONFIG={KUBECONFIG} helm list -n {KUBE_NAMESPACE}
    !KUBECONFIG={KUBECONFIG} kubectl get pods -n {KUBE_NAMESPACE}
    !KUBECONFIG={KUBECONFIG} kubectl get pods -n {KUBE_PROC_NAMESPACE}

NAME	NAMESPACE      	REVISION	UPDATED                                	STATUS  	CHART         	APP VERSION
test	dp-yanda-callan	1       	2022-10-03 06:49:39.264244447 +0000 UTC	deployed	ska-sdp-0.11.2	0.11.2     
NAME                          READY   STATUS      RESTARTS      AGE
databaseds-tango-base-0       1/1     Running     1 (32s ago)   72s
ska-sdp-console-0             1/1     Running     0             72s
ska-sdp-etcd-0                1/1     Running     0             72s
ska-sdp-helmdeploy-0          1/1     Running     0             72s
ska-sdp-lmc-config--1-5rjdk   0/1     Completed   0             72s
ska-sdp-lmc-controller-0      1/1     Running     0             72s
ska-sdp-lmc-subarray-01-0     1/1     Running     0             72s
ska-sdp-opinterface-0         1/1     Running     0             72s
ska-sdp-proccontrol-0         1/1     Running     0             72s
ska-tango-base-tangodb-0      1/1     Running     0             72s
No resources found in dp-yanda-callan-p 

In [10]:
# tango device
d = DeviceProxy('test-sdp/subarray/01')
d.set_logging_level(5)

# sdp config
config = ska_sdp_config.Config()

# ska logging
logger = logging.getLogger(__name__)
ska_ser_logging.configure_logging(level=logging.INFO)

In [13]:
# list of available workflows
!ska-sdp list script

Keys with prefix /script: 
/script/batch:test-batch:0.3.0
/script/batch:test-daliuge:0.3.0
/script/batch:test-dask:0.3.0
/script/realtime:pss-receive:0.3.0
/script/realtime:test-realtime:0.3.0
/script/realtime:test-realtime:0.4.0
/script/realtime:test-receive-addresses:0.4.0
/script/realtime:test-receive-addresses:0.5.0
/script/realtime:vis-receive:0.5.0
/script/realtime:vis-receive:0.5.1
/script/realtime:vis-receive:0.6.0
